In [ ]:
import google.generativeai as genai
from google.colab import userdata

In [ ]:
GEMINI_API_KEY=userdata.get('gemini_api_key')
genai.configure(api_key=GEMINI_API_KEY)

In [ ]:
try:
    import speech_recognition as sr
    import pyaudio
except ImportError:
    !pip install SpeechRecognition
    !apt-get install -q portaudio19-dev python3-pyaudio
    !pip install pyaudio
    import speech_recognition as sr
    import pyaudio
r = sr.Recognizer()


Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc python-pyaudio-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev python3-pyaudio
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 213 kB of archives.
After this operation, 1,043 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-pyaudio amd64 0.2.11-1.3ubuntu1 [25.9 kB]
Fetched 213 kB in 1s (371 kB/s)
Selecting previously unselected package libportau

In [ ]:
with sr.Microphone() as source:
    print("Say something!")
    audio = r.listen(source)

try:
    text = r.recognize_google(audio)
    print("You said: " + text)
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Google Speech Recognition service; {0}".format(e))

OSError: No Default Input Device Available

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash-latest')

In [ ]:
print("type 'q' to quit chat")
inp = "You are an AI assistant that will help people understand their insurance and civil policies. You also need to give simple answers so that people can understand easily. No high level terms."
while inp != 'q':
  response = model.generate_content(inp)
  print(response.text)
  inp = input("You: ").lower()

In [ ]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [61]:
user_input = inp = input("You: ").lower()

You: Hi, I am a female entrepreneur are there any policies for me


In [62]:
import json
with open("policies2-data.json", "r") as f:
    policies = json.load(f)

policy_ls = []
for i in policies.keys():
  policy_ls.append(policies[i])


In [63]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
user_embedding = model.encode(user_input)
policy_embeddings = [model.encode(policy) for policy in policy_ls]

# Calculate cosine similarity between user input and policies
similarities = cosine_similarity([user_embedding], policy_embeddings).flatten()

# Find the most relevant policy
best_policy_idx = np.argmax(similarities)
print(best_policy_idx)
best_policy = policy_ls[best_policy_idx]

5


In [64]:
best_policy

"WBCs provide free, to low-cost counseling and training and focus on women who want to start, grow, and expand their small business.Women's Business Centers (WBCs) are a part a national network of entrepreneurship centers throughout the United States and its territories, which are designed to assist women in starting and growing small businesses. WBCs seek to level the playing field for women entrepreneurs, who still face unique obstacles in the business world.SBA helps women entrepreneurs launch new businesses and compete in the marketplace. There are a number of resources for women-owned businesses including business training, counseling, federal contracts, and access to credit and capital."

In [65]:
prompt = f"""
You are an AI assistant that helps people understand insurance and civil policies.
The user has asked the following question: "{user_input}"

Here is the most relevant policy:
Policy Description: {best_policy}

Please respond in a simple and easy-to-understand way, avoiding complex terms.
"""
gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest') # Your API call function here
# response = gemini_model.generate_content(inp)
# Display Gemini's response to the user
# print(f"Gemini's Response: {response}")

In [66]:
response = gemini_model.generate_content(prompt)



2024-09-29 16:24:00.007 200 POST /v1beta/models/gemini-1.5-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1646.81ms


In [ ]:
response.candidates[0].content.parts[0].text

"Hey there! It's great you're looking into scholarships for college. \n\nThe Coca-Cola First Generation Scholarship is a great option for students like you who are the first in their family to go to college. This scholarship offers up to $5,000 to help you pay for your studies. \n\nHere's the key information you need to know:\n\n**Who can apply:**\n\n* You must be a full-time student at a UNCF college (check the list to see if your school is included).\n* You must be the first person in your family to go to a 4-year college.\n* You need to have a good academic record (GPA of 3.0 or higher).\n* You need to show that you have financial need. \n\n**What to do:**\n\n* Apply online by September 30th, 2024.\n* Submit your transcript and write an essay.\n\n**Important Note:**  This scholarship is NOT available for students at Morehouse College, Spelman College, Clark Atlanta University, or the Interdenominational Theological Center.\n\nI hope this helps you understand this scholarship opportu

In [ ]:
# !pip install gradio

In [67]:
import gradio as gr

def echo(message, history):
    user_input = message
    similarities = cosine_similarity([user_embedding], policy_embeddings).flatten()
    best_policy_idx = np.argmax(similarities)
    print(best_policy_idx)
    best_policy = policy_ls[best_policy_idx]
    prompt = f"""
You are an AI assistant that helps people understand insurance and civil policies.
The user has asked the following question: "{user_input}"

Here is the most relevant policy:
Policy Description: {best_policy}

Please respond in a simple and easy-to-understand way, avoiding complex terms.
"""
    gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest')

    print("=== Prompt==\n\n", prompt)
    response = gemini_model.generate_content(prompt)
    print("\n\n=== Response == ",response.candidates[0].content.parts[0].text)
    response = str(response.candidates[0].content.parts[0].text)

    return response

demo = gr.ChatInterface(fn=echo, title="Policy AI")
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://624d17922115d54064.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


5
=== Prompt==

 
You are an AI assistant that helps people understand insurance and civil policies.
The user has asked the following question: "I am a female entreprenuer, looking for any policies that might help me grow my buisness"

Here is the most relevant policy:
Policy Description: WBCs provide free, to low-cost counseling and training and focus on women who want to start, grow, and expand their small business.Women's Business Centers (WBCs) are a part a national network of entrepreneurship centers throughout the United States and its territories, which are designed to assist women in starting and growing small businesses. WBCs seek to level the playing field for women entrepreneurs, who still face unique obstacles in the business world.SBA helps women entrepreneurs launch new businesses and compete in the marketplace. There are a number of resources for women-owned businesses including business training, counseling, federal contracts, and access to credit and capital.

Please r

2024-09-29 16:24:51.578 200 POST /v1beta/models/gemini-1.5-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1517.93ms




=== Response ==  That's great! You're looking to grow your business, and there are resources specifically designed to help women entrepreneurs like you.

One option is to look for a **Women's Business Center (WBC)**. These centers offer free or low-cost training and advice to women starting or growing their businesses. They're like a support network for women entrepreneurs, helping them overcome the unique challenges they face.

Another resource is the **Small Business Administration (SBA)**. They offer programs and resources specifically for women-owned businesses, including:

* **Business training:** Learn valuable skills to improve your business.
* **Counseling:** Get expert advice and guidance on your business decisions.
* **Federal contracts:**  Get opportunities to work with the government.
* **Access to credit and capital:** Find ways to get funding to help your business grow.

Finding these resources is a great way to get the support and guidance you need to build a successfu

2024-09-29 16:25:30.315 200 POST /v1beta/models/gemini-1.5-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1190.06ms




=== Response ==  While there isn't a specific policy *just* for farmers, there are programs that can help women who are farmers or want to start a farm business. 

The Women's Business Centers (WBCs) offer free or low-cost advice and training to help women entrepreneurs. They are part of a nationwide network that wants to make it easier for women to start and grow their businesses.  

The Small Business Administration (SBA) also has programs to help women-owned businesses, including training, advice, and access to loans.  

So, even though there isn't a specific "farmer's policy," these programs can help women who are farmers or who want to be! 

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7865 <> https://624d17922115d54064.gradio.live
